In [0]:
#load history data

from pyspark.sql.functions import lit,now,to_date,xxhash64

customer= spark.read.parquet("dbfs:/mnt/ecom1/customer/customers.parquet")  
spark.sql("create database if not exists customer_master;")
customer.write.mode("overwrite").saveAsTable("customer_master.customers")

In [0]:
%sql
describe extended customer_master.customers


col_name,data_type,comment
id,bigint,null
customer_id,string,null
customer_city,string,null
customer_state,string,null
customer_zip_code_prefix,string,null
,,
# Delta Statistics Columns,,
Column Names,"customer_id, customer_state, customer_zip_code_prefix, id, customer_city",
Column Selection Method,first-32,
,,


In [0]:
%sql
select * from customer_master.customers 
 where customer_id in('861eff4711a542e4b93843c6dd7febb0','290c77bc529b7ac935b93aa66c333dc3','9999999999999999999999999990')
order by customer_id



id,customer_id,customer_city,customer_state,customer_zip_code_prefix
2,290c77bc529b7ac935b93aa66c333dc3,sao bernardo do campo,SP,9790
1,861eff4711a542e4b93843c6dd7febb0,franca,SP,14409


In [0]:


#scd type 1

from pyspark.sql.functions import coalesce  
customer_d= spark.read.parquet("dbfs:/mnt/ecom1/customer/customer_delta.parquet")


customer_d.display()

customer_id,customer_zip_code_prefix,customer_city,customer_state
861eff4711a542e4b93843c6dd7febb0,15000,franca,SP
290c77bc529b7ac935b93aa66c333dc3,9790,sao paulo,SP
9999999999999999999999999990,81560,timoteo,MG


In [0]:
update_ds=customer.join(customer_d,"customer_id","full_outer")\
    .select(coalesce(customer.customer_id,customer_d.customer_id).alias("customer_id"),
            coalesce(customer_d.customer_zip_code_prefix,customer.customer_zip_code_prefix).alias("customer_zip_code_prefix"),
            coalesce(customer_d.customer_city,customer.customer_city).alias("customer_city"),
            coalesce(customer_d.customer_state,customer.customer_state).alias("customer_state"),
            )
    

update_ds.write.mode("overwrite").saveAsTable("customer_master.customers")


In [0]:
%sql
select * from customer_master.customers where customer_id in('861eff4711a542e4b93843c6dd7febb0','290c77bc529b7ac935b93aa66c333dc3','9999999999999999999999999990')


id,customer_id,customer_city,customer_state,customer_zip_code_prefix
null,861eff4711a542e4b93843c6dd7febb0,franca,SP,15000
null,290c77bc529b7ac935b93aa66c333dc3,sao paulo,SP,9790
null,9999999999999999999999999990,timoteo,MG,81560


In [0]:

#SCD 1 with merge
from delta.tables import *
cust_tbl_inst = DeltaTable.forPath(spark, 'dbfs:/user/hive/warehouse/customer_master.db/customers')
customer_d= spark.read.parquet("dbfs:/mnt/ecom1/customer/customer_delta.parquet")

cust_tbl_inst.alias("trg").merge(customer_d.alias("src"),condition="trg.customer_id=src.customer_id"

                       ).whenMatchedUpdate(set ={
                         
"customer_zip_code_prefix": "src.customer_zip_code_prefix",
"customer_city": "src.customer_city",
"customer_state": "src.customer_state"

                       }

                        ).whenNotMatchedInsert(values={
"customer_id": "customer_id",
"customer_zip_code_prefix": "customer_zip_code_prefix",
"customer_city": "customer_city",
"customer_state": "customer_state"}).execute()
